# Algoritmo branch-and-bound

---

Este projeto consiste na criação de um programa para resolver problemas de maximização em programação linear inteira binária. Utilizaremos o algoritmo Branch-and-Bound para encontrar a solução ótima. 

O objetivo é encontrar o valor máximo de uma função sujeita a diversas restrições, todas do tipo "menor ou igual". 

O algoritmo irá dividir o problema em subproblemas menores e, através de uma estratégia de ramificação, explorará diferentes soluções até encontrar a ótima. A Estratégia de ramificação usada será a: <br> <br>

<div style="border:2px solid black; padding: 10px; background-color:#CEE0DC">
Busca em profundidade: para isso, utilizamos uma pilha para armazenar os nós abertos (nós que ainda não foram processados)</div>

Para resolver os subproblemas, faremos uso de um pacote de programação linear, como o python-mip. 

---

In [180]:
from mip import Model, MAXIMIZE, xsum
from numpy import asarray, abs
import re

In [181]:
def ler_txt(caminho_arquivo: str):
    l = []
    with open(caminho_arquivo) as arquivo:
        for linha in arquivo:
            for i in re.findall(r'\d+', linha):
                l.append(i)

    variaveis = int(l[0])
    restricoes = int(l[1])
    coef_obj = []
    for i in range(2, 2 + variaveis):
        coef_obj.append(int(l[i]))

    count = 0
    aux = []
    coef_restricoes = []
    for i in range(2 + variaveis, len(l)):
        if count == variaveis:
            aux.append(int(l[i]))
            coef_restricoes.append(aux)
            aux = []
            count = 0
        else:
            aux.append(int(l[i]))
            count += 1
            
    return variaveis, restricoes, coef_obj, coef_restricoes

Exemplo teste1.txt:

$$
\text{max } Z = 2 \quad 10 \quad 8 \quad 7 \quad 10 \quad 10 \quad 6\\
$$

s.a:
\begin{gather*}
7 \quad 11\\
2 \quad 10 \quad 8 \quad 7 \quad 10 \quad 10 \quad 6\\
5 \quad 7 \quad 8 \quad 1 \quad 7 \quad 5 \quad 6 \quad 20\\
1 \quad 6 \quad 4 \quad 9 \quad 10 \quad 6 \quad 10 \quad 30\\
4 \quad 4 \quad 4 \quad 1 \quad 5 \quad 5 \quad 10 \quad 40\\
3 \quad 10 \quad 8 \quad 1 \quad 3 \quad 3 \quad 8 \quad 30\\
10 \quad 8 \quad 9 \quad 9 \quad 7 \quad 6 \quad 10 \quad 20\\
6 \quad 6 \quad 3 \quad 6 \quad 3 \quad 7 \quad 2 \quad 80\\
7 \quad 10 \quad 7 \quad 8 \quad 7 \quad 8 \quad 7 \quad 100\\
9 \quad 8 \quad 1 \quad 1 \quad 8 \quad 10 \quad 2 \quad 90\\
1 \quad 5 \quad 3 \quad 10 \quad 2 \quad 4 \quad 9 \quad 70\\
9 \quad 6 \quad 1 \quad 4 \quad 7 \quad 5 \quad 10 \quad 60\\
5 \quad 7 \quad 4 \quad 4 \quad 3 \quad 4 \quad 10 \quad 40\\
\end{gather*}


In [182]:
def resolver(modelo: Model):
    modelo.optimize()
    parametros = {}
    parametros["objetivo"] = modelo.objective_value
    parametros["variaveis"] = modelo.vars
    return parametros

In [200]:
def podar(modelo: Model, primal):
    """
    Verifica se um nó na árvore de busca pode ser podado com base em diferentes critérios.

    Args:
        modelo: O modelo a ser avaliado.
        primal: O valor primal atual.

    Returns:
        str: Um dos seguintes valores: 'INVIAVEL', 'INTEIRO', 'LIMITANTE' ou 'VIÁVEL'.
    """
    
    aux_solver = resolver(modelo)
    count_int = 0

    if aux_solver["objetivo"] == None:
        return 'INVIAVEL'

    for i in aux_solver["variaveis"]:
        if i.x.is_integer():
            count_int += 1

    if count_int == len(aux_solver["variaveis"]):
        return 'INTEIRO'

    if aux_solver["objetivo"] <= primal:
        return 'LIMITANTE'
    
    return 'VIÁVEL'

In [184]:
def encontrar_mais_proximo(array: list, valor: float):
    
    array = asarray(array)
    idx = (abs(array - valor)).argmin()
    return idx

In [201]:
def ramificar(modelo: Model, valores_solucao):
    
    var_ramificacao = valores_solucao[encontrar_mais_proximo([i.x for i in valores_solucao], 0.5)]

    modelo_0 = modelo.copy()
    modelo_0 += var_ramificacao == 0

    modelo_1 = modelo.copy()
    modelo_1 += var_ramificacao == 1

    return (modelo_0, modelo_1)

In [202]:
def branch_and_bound(modelo: Model):
    global primal
    global modelo_otimo
    
    primal = 0
    modelo_otimo = None
    
    nos = [modelo]

    while nos != []:
        
        modelo_solver = resolver(nos[0])
        aux = podar(nos[0], primal)
        if aux == 'INVIAVEL' or aux == 'LIMITANTE':
            nos.pop(0)
            
        elif aux == 'INTEIRO':
            if modelo_solver["objetivo"] >= primal:
                modelo_otimo = nos[0]
                primal = modelo_solver["objetivo"]
            nos.pop(0)
            
        elif aux == 'VIÁVEL':
            ramos = ramificar(nos[0], modelo_solver["variaveis"])
            nos.append(ramos[0])
            nos.append(ramos[1])
            nos.pop(0)

In [203]:
def executar(caminho_arquivo):
    
    variaveis, restricoes, coef_obj, coef_restricoes = ler_txt(caminho_arquivo)

    modelo = Model(sense=MAXIMIZE)

    x = [modelo.add_var(var_type="CONTINUOUS", lb=0, ub=1, name="x_" + str(i)) for i in range(variaveis)]

    modelo.objective = xsum(coef_obj[i] * x[i] for i in range(variaveis))

    for i in range(restricoes):
        modelo += xsum(coef_restricoes[i][j] * x[j] for j in range(variaveis)) <= coef_restricoes[i][-1]

    branch_and_bound(modelo)

    imprimir_resultado(resolver(modelo_otimo))

In [204]:
def imprimir_resultado(modelo_resolvido):
    
    print("Variáveis:")
    for i in modelo_resolvido["variaveis"]:
        print(i.name, ' = ', i.x)
    print("Função objetivo:")
    print('Z = ', modelo_resolvido["objetivo"])

In [205]:
executar("teste1.txt")

Variáveis:
x_0  =  0.0
x_1  =  1.0
x_2  =  0.0
x_3  =  0.0
x_4  =  1.0
x_5  =  0.0
x_6  =  0.0
Função objetivo:
Z =  20.0


In [174]:
executar("teste2.txt")

Variáveis:
x_0  =  0.0
x_1  =  1.0
x_2  =  0.0
x_3  =  0.0
x_4  =  0.0
x_5  =  1.0
x_6  =  1.0
x_7  =  0.0
x_8  =  0.0
Função objetivo:
Z =  24.0


In [175]:
executar("teste3.txt")

Variáveis:
x_0  =  0.0
x_1  =  0.0
x_2  =  1.0
x_3  =  0.0
x_4  =  0.0
x_5  =  0.0
x_6  =  1.0
x_7  =  0.0
x_8  =  0.0
Função objetivo:
Z =  19.0


In [176]:
executar("teste4.txt")

Variáveis:
x_0  =  0.0
x_1  =  0.0
x_2  =  1.0
x_3  =  0.0
x_4  =  0.0
x_5  =  0.0
x_6  =  0.0
x_7  =  0.0
x_8  =  0.0
Função objetivo:
Z =  10.0
